In [1]:
# importamos las librerias necesarias

import re
import os
import xml.etree.ElementTree as ET
import mysql.connector

In [3]:
# Creamos la clase para abrir y limpiar los ficheros .xml, así como introducir sus datos a la base de datos
class Ficheros_xml: 
    def __init__(self, ruta):
        self.ruta = ruta


    def abrir_xml(self, fichero_xml):
        self.fichero_xml = fichero_xml 
        # Abrimos el fichero
        tree = ET.parse(f'{self.ruta}/{self.fichero_xml}.xml')
        root = tree.getroot()
        # Extraemos el contenido del fichero y lo appendeamos a una lista
        xml_list = []
        for child in root:
            xml_dict = {}
            for subchild in child:
                xml_dict[subchild.tag] = subchild.text
            xml_list.append(xml_dict)
        return xml_list  # Obtenemos una lista de diccionarios
    

    def limpiar_xml(self, xml_list):
        self.xml_list = xml_list
        # Eliminamos el elemento de key 'level_0' de todos los diccionarios porque es redundante
        for diccionario in self.xml_list:
            if 'level_0' in diccionario:
                diccionario.pop('level_0')
            else:
                pass
        # Convertimos los valores de la key 'index' a integer:
        for diccionario in self.xml_list:
            diccionario['index'] = int(diccionario['index'])
        # La variable genéro esta codificada y deberemos cambiarla:
        for diccionario in self.xml_list:
            if 'gender' in diccionario:
                if diccionario['gender'] == '0':
                    diccionario['gender'] = 'Man'
                elif diccionario['gender'] == '1':
                    diccionario['gender'] = 'Woman'
                elif diccionario['gender'] == '2':
                    diccionario['gender'] = 'Nonbinary'        
                elif diccionario['gender'] == '3':
                    diccionario['gender'] = 'Prefer not to say'  
                elif diccionario['gender'] == '4':
                    diccionario['gender'] = 'Prefer to self-describe'
                else:
                    print('El valor es no válido')              
            else:
                pass
        # Creamos los valores para la columna index_sql
        for indice, diccionario in enumerate(self.xml_list):
            diccionario.setdefault('index_sql', (indice+1))
        # Convertimos en resultado en una lista de tuplas
        lista_tuplas_xml = []
        for dicc in self.xml_list:
            tupla = tuple(dicc.values())
            lista_tuplas_xml.append(tupla)
        return lista_tuplas_xml


    def insercion_valores_xml(self, val, **datos_acceso):
        self.val = val
        self.datos_acceso = datos_acceso
        cnx = mysql.connector.connect(**self.datos_acceso)
        mycursor = cnx.cursor()
        query = "INSERT INTO data_xml (index_xml, time, age, gender, index_sql) VALUES (%s, %s, %s, %s, %s)"
        try: 
            mycursor.executemany(query, self.val)
            cnx.commit()
            return f'{mycursor.rowcount} registro/s insertado/s.'
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

In [4]:
# Creamos las variables con los argumentos
mi_ruta = 'datos'
nombre_fichero_xml = 'data_xml'

In [5]:
# Instanciamos la clase primero para poder utilizar los métodos
instancia_xml = Ficheros_xml(mi_ruta)

In [6]:
# Guardamos el contenido del fichero .xml en una variable
xml_original = instancia_xml.abrir_xml(nombre_fichero_xml)
xml_original

[{'level_0': '1', 'index': '1', 'time': '784', 'age': '50-54', 'gender': '0'},
 {'level_0': '2', 'index': '2', 'time': '924', 'age': '22-24', 'gender': '0'},
 {'level_0': '3', 'index': '3', 'time': '575', 'age': '45-49', 'gender': '0'},
 {'level_0': '4', 'index': '4', 'time': '781', 'age': '45-49', 'gender': '0'},
 {'level_0': '5', 'index': '5', 'time': '1020', 'age': '25-29', 'gender': '1'},
 {'level_0': '6', 'index': '6', 'time': '141', 'age': '18-21', 'gender': '1'},
 {'level_0': '7', 'index': '7', 'time': '484', 'age': '30-34', 'gender': '0'},
 {'level_0': '8', 'index': '8', 'time': '1744', 'age': '22-24', 'gender': '0'},
 {'level_0': '9', 'index': '9', 'time': '655', 'age': '30-34', 'gender': '0'},
 {'level_0': '10',
  'index': '10',
  'time': '1777',
  'age': '40-44',
  'gender': '0'},
 {'level_0': '11',
  'index': '11',
  'time': '3081',
  'age': '18-21',
  'gender': '1'},
 {'level_0': '12',
  'index': '12',
  'time': '1922',
  'age': '18-21',
  'gender': '1'},
 {'level_0': '13'

In [7]:
# Limpiamos los datos del archivo .xml
xml_limpio = instancia_xml.limpiar_xml(xml_original)
xml_limpio

[(1, '784', '50-54', 'Man', 1),
 (2, '924', '22-24', 'Man', 2),
 (3, '575', '45-49', 'Man', 3),
 (4, '781', '45-49', 'Man', 4),
 (5, '1020', '25-29', 'Woman', 5),
 (6, '141', '18-21', 'Woman', 6),
 (7, '484', '30-34', 'Man', 7),
 (8, '1744', '22-24', 'Man', 8),
 (9, '655', '30-34', 'Man', 9),
 (10, '1777', '40-44', 'Man', 10),
 (11, '3081', '18-21', 'Woman', 11),
 (12, '1922', '18-21', 'Woman', 12),
 (13, '852', '45-49', 'Man', 13),
 (14, '838', '22-24', 'Man', 14),
 (15, '563', '35-39', 'Man', 15),
 (16, '1315', '50-54', 'Man', 16),
 (17, '479', '18-21', 'Man', 17),
 (18, '249', '22-24', 'Man', 18),
 (19, '650', '30-34', 'Man', 19),
 (20, '1461', '70+', 'Man', 20),
 (21, '551', '25-29', 'Woman', 21),
 (22, '258', '30-34', 'Man', 22),
 (23, '773', '35-39', 'Man', 23),
 (24, '630', '18-21', 'Man', 24),
 (25, '2467', '25-29', 'Woman', 25),
 (26, '7750', '45-49', 'Man', 26),
 (27, '607', '22-24', 'Man', 27),
 (28, '525', '22-24', 'Woman', 28),
 (29, '501', '18-21', 'Man', 29),
 (30, '415'

In [8]:
# Creamos un diccionario con los datos de acceso a la base de datos
config = {
        'user': 'root', 
        'password': 'AlumnaAdalab',
        'host': '127.0.0.1',
        'database': 'proyecto1',
        'raise_on_warnings': True
}

In [9]:
# Insertamos los datos del archivo xml a la tabla de la base de datos:
instancia_xml.insercion_valores_xml(xml_limpio, **config)

'25972 registro/s insertado/s.'